# Extend colmuns

In [1]:
import os
import json
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt
%matplotlib inline
import lightgbm as lgb

In [2]:
# dirs
input_dir = '../input/'
working_dir = '../working/'
output_dir = '../output/'

In [3]:
# https://www.kaggle.com/julian3833/1-quick-start-read-csv-and-flatten-json-fields

def load_df(csv_path=os.path.join(input_dir, 'train.csv'), nrows=None):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    
    df = pd.read_csv(csv_path, 
                     converters={column: json.loads for column in JSON_COLUMNS}, 
                     dtype={'fullVisitorId': 'str'}, # Important!!
                     nrows=nrows)
    
    for column in JSON_COLUMNS:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
    print(f"Loaded {os.path.basename(csv_path)}. Shape: {df.shape}")
    
    # shorten col names
    short_col_names = [i.split('.')[-1] for i in df.columns]
    df.columns = short_col_names
    return df

In [4]:
train = load_df(csv_path=os.path.join(input_dir, 'train.csv'))
test = load_df(csv_path=os.path.join(input_dir, 'test.csv'))

Loaded train.csv. Shape: (903653, 55)
Loaded test.csv. Shape: (804684, 53)


In [5]:
train.head()

,channelGrouping,date,fullVisitorId,sessionId,socialEngagementType,visitId,visitNumber,visitStartTime,browser,browserSize,...,isVideoAd,page,slot,campaign,campaignCode,isTrueDirect,keyword,medium,referralPath,source
0,Organic Search,20160902,1131660440785968503,1131660440785968503_1472830385,Not Socially Engaged,1472830385,1,1472830385,Chrome,not available in demo dataset,...,NaN,NaN,NaN,(not set),NaN,NaN,(not provided),organic,NaN,google
1,Organic Search,20160902,377306020877927890,377306020877927890_1472880147,Not Socially Engaged,1472880147,1,1472880147,Firefox,not available in demo dataset,...,NaN,NaN,NaN,(not set),NaN,NaN,(not provided),organic,NaN,google
2,Organic Search,20160902,3895546263509774583,3895546263509774583_1472865386,Not Socially Engaged,1472865386,1,1472865386,Chrome,not available in demo dataset,...,NaN,NaN,NaN,(not set),NaN,NaN,(not provided),organic,NaN,google
3,Organic Search,20160902,4763447161404445595,4763447161404445595_1472881213,Not Socially Engaged,1472881213,1,1472881213,UC Browser,not available in demo dataset,...,NaN,NaN,NaN,(not set),NaN,NaN,google + online,organic,NaN,google
4,Organic Search,20160902,27294437909732085,27294437909732085_1472822600,Not Socially Engaged,1472822600,2,1472822600,Chrome,not available in demo dataset,...,NaN,NaN,NaN,(not set),NaN,True,(not provided),organic,NaN,google


In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 903653 entries, 0 to 903652
Data columns (total 55 columns):
channelGrouping              903653 non-null object
date                         903653 non-null int64
fullVisitorId                903653 non-null object
sessionId                    903653 non-null object
socialEngagementType         903653 non-null object
visitId                      903653 non-null int64
visitNumber                  903653 non-null int64
visitStartTime               903653 non-null int64
browser                      903653 non-null object
browserSize                  903653 non-null object
browserVersion               903653 non-null object
deviceCategory               903653 non-null object
flashVersion                 903653 non-null object
isMobile                     903653 non-null bool
language                     903653 non-null object
mobileDeviceBranding         903653 non-null object
mobileDeviceInfo             903653 non-null object
mobileDevic

In [7]:
train['visitId'].value_counts().head()

1493146175    8
1484649802    6
1478345904    6
1481369525    6
1494374199    5
Name: visitId, dtype: int64

In [8]:
sample_submission = pd.read_csv(os.path.join(input_dir, 'sample_submission.csv'))
sample_submission.shape

(617242, 2)

In [9]:
test['fullVisitorId'].value_counts().shape

(617242,)

In [10]:
sample_submission['fullVisitorId'].value_counts().shape

(617242,)

In [13]:
train.iloc[0,:].values

array(['Organic Search', 20160902, '1131660440785968503',
       '1131660440785968503_1472830385', 'Not Socially Engaged',
       1472830385, 1, 1472830385, 'Chrome',
       'not available in demo dataset', 'not available in demo dataset',
       'desktop', 'not available in demo dataset', False,
       'not available in demo dataset', 'not available in demo dataset',
       'not available in demo dataset', 'not available in demo dataset',
       'not available in demo dataset', 'not available in demo dataset',
       'Windows', 'not available in demo dataset',
       'not available in demo dataset', 'not available in demo dataset',
       'Izmir', 'not available in demo dataset', 'Asia', 'Turkey',
       'not available in demo dataset', 'not available in demo dataset',
       '(not set)', 'ttnet.com.tr', 'not available in demo dataset',
       'Izmir', 'Western Asia', '1', '1', '1', '1', nan, '1', nan, nan,
       'not available in demo dataset', nan, nan, nan, nan, '(not set)',
     

In [12]:
train.to_csv(os.path.join(input_dir, 'train_expand.csv'), index=False)
test.to_csv(os.path.join(input_dir, 'test_expand.csv'), index=False)